In [ ]:
import zmq
from brainflow.board_shim import BoardIds, BrainFlowInputParams, BoardShim
import time
import random
import os
import json  

# Initialize BrainFlowInputParams for the Synthetic Board
params = BrainFlowInputParams()
params.serial_port = "/dev/ttyUSB0"  # Replace with any dummy string, as it's not used by the Synthetic Board

# Create a BoardShim instance for the Synthetic Board
board = BoardShim(BoardIds.SYNTHETIC_BOARD, params)

# Number of simulated ECoG channels
num_ecog_channels = 32

# Sampling frequency in Hz
sampling_frequency = 500

# Calculate the time interval for data generation
time_interval = 1 / sampling_frequency

# Define the voltage range (1 microvolt to 8 microvolts)
min_volt = 1e-6  # 1 microvolt
max_volt = 200e-6  # 200 microvolts

# Initialize ZeroMQ publisher socket
context = zmq.Context()
socket = context.socket(zmq.PUB)
socket.bind("tcp://*:5444")  # Publish on port 5444

# Initialize a variable to store the last print time
last_print_time = time.time()

# Define a loop to continuously generate synthetic ECoG data and publish it
while True:
    # Record the start time for timing purposes
    start_time = time.time()

    # Simulate ECoG data for each channel within the voltage range
    ecog_data = [random.uniform(min_volt, max_volt) for _ in range(num_ecog_channels)]

    # Serialize the ECoG data as JSON and encode it for transmission
    ecog_data_json = json.dumps(ecog_data).encode('utf-8')

    # Publish the serialized ECoG data to the ZeroMQ socket
    socket.send(ecog_data_json)

    # Calculate the time difference since the last print
    time_difference = time.time() - last_print_time

    # If 10 seconds have passed since the last print, print the data for all channels
    if time_difference >= 10:
        # Clear the console screen before printing new data
        os.system('cls' if os.name == 'nt' else 'clear')

        print("--------------------")
        print("ECoG data for all channels:")
        for channel, data in enumerate(ecog_data, start=1):
            print(f"Channel {channel}: {data}")
        print("--------------------")

        # Update the last print time to the current time
        last_print_time = time.time()

    # Calculate the time taken for generating and publishing data
    elapsed_time = time.time() - start_time

    # Sleep for the remaining time to maintain a continuous rate of 500 Hz
    if elapsed_time < time_interval:
        time.sleep(time_interval - elapsed_time)
